In [ ]:
import itertools
import os
import pylhe
pylhe.register_awkward()

lhe_file = "test_pp_mumu/Events/run_01/unweighted_events.lhe"

# check if file is needs to be unzipped
if os.path.isfile(f"{lhe_file}.gz"):
    os.system(f"gunzip {lhe_file}.gz")

events = pylhe.read_lhe_with_attributes(lhe_file)
n_events = pylhe.read_num_events(lhe_file)
print(f"Number of events: {n_events}")


In [ ]:
event_array = pylhe.to_awkward(pylhe.read_lhe_with_attributes(lhe_file))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import mplhep as hep
plt.style.use(hep.style.ROOT)

outgoing_muon_mask = (event_array["particles"]["id"] == 13) & (event_array["particles"]["status"] == 1)
outgoing_antimuon_mask = (event_array["particles"]["id"] == -13) & (event_array["particles"]["status"] == 1)
z = event_array["particles"]["vector"][outgoing_muon_mask] + event_array["particles"]["vector"][outgoing_antimuon_mask]

In [ ]:
plt.figure()
plt.subplot(211)
plt.hist(z.pt, bins=np.linspace(0, 120, 100), label='MadGraph')
plt.xlabel("$p_{T}(Z)$ [GeV]")
plt.ylabel("Events")
plt.legend()
plt.subplot(212)
plt.hist(z.m, bins=np.linspace(60, 120, 100), label='MadGraph')
plt.xlabel("$m(Z)$ [GeV]")
plt.ylabel("Events")
plt.legend()
plt.tight_layout()
plt.savefig("madgraph.pdf")

In [ ]:
import pyhepmc
with pyhepmc.open("test_pp_mumu_pythia/Events/run_01/tag_1_pythia8_events.hepmc") as f:
    event = f.read()
event

In [ ]:
import numpy as np
z_m = []
z_pt = []
with pyhepmc.open("test_pp_mumu_pythia/Events/run_01/tag_1_pythia8_events.hepmc") as f:
    for event in f:
        for p in event.particles:
            if p.pid == 13 and p.status == 1:
                muon = p.momentum
            if p.pid == -13 and p.status == 1:
                antimuon = p.momentum
        z = muon + antimuon
        z_m.append(z.m() / 1000)
        z_pt.append(z.pt() / 1000)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import mplhep as hep
plt.style.use(hep.style.ROOT)
plt.figure()
plt.subplot(211)
plt.hist(z_pt, bins=np.linspace(0, 120, 100), label='MadGraph+Pythia')
plt.xlabel("$p_{T}(Z)$ [GeV]")
plt.ylabel("Events")
plt.legend()
plt.subplot(212)
plt.hist(z_m, bins=np.linspace(60, 120, 100), label='MadGraph+Pythia')
plt.xlabel("$m(Z)$ [GeV]")
plt.ylabel("Events")
plt.legend()
plt.tight_layout()
plt.savefig("madgraph_p_pythia.pdf")